# Job Hunt Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/job_hunt_agents.ipynb)

## Dependencies

In [16]:
# Install dependencies without output
%pip install langchain_community > /dev/null
%pip install praisonai[crewai] > /dev/null
%pip install requests > /dev/null


## Tools

In [25]:
#ToDo: Test Pending, Test with Adzuna API Key
import json, requests, os
from praisonai_tools import BaseTool

class JobSearchTool(BaseTool):
    name: str = "JobSearchTool"
    description: str = (
        "This tool searches for job listings using the Adzuna API.\n"
        "Provide JSON input with 'role', 'location', and 'num_results'.\n"
        "Example:\n"
        "   - Input: {'role': 'Data Scientist', 'location': 'New York', 'num_results': 5}\n"
        "   - Output: List of job titles, companies, locations, and brief descriptions."
    )

    def _run(self, input_json: str) -> str:
        # Parse input JSON string
        try:
            input_data = json.loads(input_json)
            role = input_data['role']
            location = input_data['location']
            num_results = input_data['num_results']
        except (json.JSONDecodeError, KeyError) as e:
            return "Invalid input format. Ensure JSON format: {'role': '<role>', 'location': '<location>', 'num_results': <number>}."

        # Adzuna API parameters
        app_id = os.getenv('ADZUNA_APP_ID')
        api_key = os.getenv('ADZUNA_API_KEY')
        base_url = "http://api.adzuna.com/v1/api/jobs"
        url = f"{base_url}/us/search/1?app_id={app_id}&app_key={api_key}&results_per_page={num_results}&what={role}&where={location}&content-type=application/json"

        try:
            response = requests.get(url)
            response.raise_for_status()
            jobs_data = response.json()

            job_listings = []
            for job in jobs_data.get('results', []):
                job_details = f"Title: {job['title']}, Company: {job['company']['display_name']}, Location: {job['location']['display_name']}, Description: {job['description'][:100]}..."
                job_listings.append(job_details)
            return '\n'.join(job_listings)
        except requests.exceptions.HTTPError as err:
            return f"HTTP Error: {err}"
        except requests.exceptions.RequestException as e:
            return f"Request Error: {e}"



## YAML Prompt

In [ ]:
agent_yaml = """
framework: "crewai"
topic: "Job Search Assistance and Career Advisory"
roles:
  job_searcher_agent:
    role: "Job Searcher"
    backstory: |
      An experienced job searcher actively seeking new opportunities in the specified field. Skilled in identifying job listings relevant to the user's qualifications.
    goal: |
      Conduct job searches for specified roles and locations, focusing on discovering relevant listings that match the user's skills and interests.
    verbose: true
    allow_delegation: true
    tools:
      - "JobSearchTool"
    tasks:
      job_search_task:
        description: |
          Search for job openings in a specified role and location using the Job Search tool. Fetch a specified number of results.
        expected_output: |
          List of relevant job listings with titles, companies, locations, and short descriptions.
  skills_development_agent:
    role: "Skills Development Advisor"
    backstory: |
      A dedicated advisor identifying key skills required for job roles and providing guidance on skill enhancement methods.
    goal: |
      Analyze job listings to identify essential skills for each position, recommending resources and methods to develop those skills.
    verbose: true
    allow_delegation: true
    tools: []
    tasks:
      skills_highlighting_task:
        description: |
          Analyze job listings for skills requirements. Provide recommendations for candidates on acquiring these skills through online courses, self-study, or hands-on experience.
        expected_output: |
          List of key skills for each job and corresponding skill development resources.
  interview_preparation_coach:
    role: "Interview Preparation Coach"
    backstory: |
      An expert in preparing candidates for interviews by simulating interview scenarios, coaching on communication skills, and providing feedback.
    goal: |
      Conduct mock interviews, focusing on enhancing the candidate’s responses, presentation, and communication.
    verbose: true
    allow_delegation: true
    tools: []
    tasks:
      interview_preparation_task:
        description: |
          Prepare candidates for interviews by conducting mock interview sessions. Offer feedback on responses, presentation, and communication skills specific to the roles applied.
        expected_output: |
          Feedback on candidate's interview responses, including strengths and areas for improvement.
  career_advisor:
    role: "Career Advisor"
    backstory: |
      Experienced in guiding job seekers through resume enhancement, LinkedIn profile optimization, and effective networking techniques.
    goal: |
      Provide comprehensive career advice, including resume building, LinkedIn profile optimization, and networking strategies.
    verbose: true
    allow_delegation: true
    tools: []
    tasks:
      career_advisory_task:
        description: |
          Offer guidance on building resumes, optimizing LinkedIn profiles, and networking strategies to strengthen job applications.
        expected_output: |
          Detailed suggestions for resume improvement, LinkedIn profile optimization, and networking strategies for job seekers.
dependencies: []
"""

## Main

In [27]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[JobSearchTool])

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 1/10


 [2024-11-02 18:45:55][DEBUG]: == Working Agent: Job Searcher
 [2024-11-02 18:45:55][INFO]: == Starting Task: Search for job openings in a specified role and location using the Job Search tool. Fetch a specified number of results.



> Entering new CrewAgentExecutor chain...
I need to conduct a job search for a specified role and location to find relevant job listings that match the user's qualifications. However, I do not have the role, location, or number of results specified yet.

Action: Ask question to coworker  
Action Input: {"question": "What specific job role and location should I search for job openings? Also, how many job listings do you want me to find?", "context": "I need to gather details about the user's job search preferences, including the specific role they are interested in, the location they want to search in, and the number of job listings to return."}  

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(




> Entering new CrewAgentExecutor chain...
I now can give a great answer.  
Final Answer: To assist you effectively in your job search, I need to gather some specific details about your preferences. Please provide the following information: 

1. **Specific Job Role**: What type of position are you interested in? This could range from entry-level roles to specialized positions in your field. If you have a particular title in mind (e.g., Marketing Coordinator, Software Developer, Project Manager), please let me know.

2. **Location**: Which geographical area are you targeting for your job search? Are you looking for opportunities in a specific city, state, or are you open to remote positions? Be as specific as possible to help narrow down the search.

3. **Number of Job Listings**: How many job listings would you like me to find for you? A common range might be between 10 to 50 listings, depending on how thorough you want your search to be.

Once I have this information, I can locate re

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(




> Entering new CrewAgentExecutor chain...
I need to gather specific preferences about the job role, location, and the number of job listings to conduct a thorough job search.

Action: None
Action Input: None

...

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

I encountered an error while trying to use the tool. This was the error: JobSearchTool._run() got an unexpected keyword argument 'input_'.
 Tool JobSearchTool accepts these inputs: JobSearchTool(input_json: 'string') - This tool searches for job listings using the Adzuna API. Provide JSON input with 'role', 'location', and 'num_results'. Example:    - Input: {'role': 'Data Scientist', 'location': 'New York', 'num_results': 5}    - Output: List of job titles, companies, locations, and brief descriptions.

Thought: I need to gather specific preferences about the job role, location, and number of job listings to conduct a thorough job search. Since I don’t have the user's preferences, I will outline the questions I need to ask.

1. What type of position are you interested in? (e.g., Marketing Coordinator, Software Developer, Project Manager)
2. Which geographical area are you targeting for your job search? (e.g., specific city, state, or remote positions)
3. How many job listings would

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

I encountered an error while trying to use the tool. This was the error: JobSearchTool._run() got an unexpected keyword argument 'input_'.
 Tool JobSearchTool accepts these inputs: JobSearchTool(input_json: 'string') - This tool searches for job listings using the Adzuna API. Provide JSON input with 'role', 'location', and 'num_results'. Example:    - Input: {'role': 'Data Scientist', 'location': 'New York', 'num_results': 5}    - Output: List of job titles, companies, locations, and brief descriptions.



KeyboardInterrupt: 